In [1]:
import numpy as np
import cv2
import cv2 as cv
from synthetic_dataset import *
from spherical_module import *

In [2]:
def draw_interest_points(img, points):
    """ Convert img in RGB and draw in green the interest points """
    img_rgb = np.stack([img, img, img], axis=2)
    for i in range(points.shape[0]):
        cv.circle(img_rgb, (points[i][0], points[i][1]), 5, (0, 255, 0), -1)
    return img_rgb

In [12]:
res_std = 256
b_img = generate_background(size=(res_std*2, res_std*4))
#b_img = generate_background(size=(1024, 2048)) /255
#cv.imshow('Random Lines', cv.resize(b_img, (1024, 512)))
#cv2.imwrite('b_img.png', b_img*255)
#cv.waitKey(0)
#cv.destroyAllWindows()

In [21]:
b_images, output_sqr = create_cube_imgs(b_img)
images = []
all_points_list = []
for idx, img in enumerate(b_images):
    if len(img.shape) == 3:
        img = img[:, :, 0]/255
    if img.shape != (res_std, res_std):
        img = cv.resize(img, (res_std, res_std))
    img *= 255
    #points = draw_polygon(img)
    #points = draw_lines(img)
    #points = draw_multiple_polygons(img)
    #points = draw_ellipses(img)
    #points = draw_star(img)
    #points = draw_checkerboard(img)
    #points = draw_stripes(img)
    #points = draw_cube(img)

    #points = draw_lines(img)
    #all_points_list.append(points)
    flag_one = 0
    if flag_one:
        if idx == 5:
            points = draw_lines(img)
            all_points_list.append(points)
            cv.imshow('Random Lines', img)
            cv2.imwrite('aaa_img.png', img*255)
            cv.waitKey(0)
            cv.destroyAllWindows()
        else:
            all_points_list.append([])
    else:
        #points = draw_lines(img)
        points = draw_polygon(img)
        all_points_list.append(points)

    #points = gaussian_noise(img)
    #print(points)
    if False:
        img = draw_interest_points(img, points)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    images.append(img)

In [22]:
# 6枚の画像を1枚のキューブマップ画像に結合する
cube_map_size = (res_std * 3, res_std * 4)  # キューブマップのサイズ
cube_map = np.zeros(cube_map_size, dtype=np.uint8)

positions = [(res_std, res_std), (res_std, 0), (res_std, res_std*3), 
             (res_std, res_std*2), (0, res_std), (res_std*2, res_std)]
for idx, (pos, img) in enumerate(zip(positions, images)):
    cube_map[pos[0]:pos[0]+res_std, pos[1]:pos[1]+res_std] = img

flag_cube_points_vis = False
if flag_cube_points_vis:
    for idx, (pos, img) in enumerate(zip(positions, images)):
        all_points_list[idx][:, 0] += pos[1]
        all_points_list[idx][:, 1] += pos[0]
    points = np.concatenate(all_points_list, axis=0)
    cube_map = draw_interest_points(cube_map, points)
    #cube_map = cv2.cvtColor(cube_map, cv2.COLOR_BGR2GRAY)
    cv.imshow('Random Lines', cv.resize(cube_map, (600, 400)))
    cv2.imwrite('cube_img.png', cube_map)
    cv.waitKey(0)
    cv.destroyAllWindows()
cv.imshow('Random Lines', cv.resize(cube_map, (600, 400)))
cv2.imwrite('cube_img.png', cube_map)
cv.waitKey(0)
cv.destroyAllWindows()

In [23]:
def cubemap_to_equirectangular_coord(face, x, y, cubemap_size=256):
    """
    Convert cubemap coordinates to equirectangular coordinates.

    Parameters:
    - face: The cubemap face index (0: front, 1: back, 2: right, 3: left, 4: top, 5: bottom).
    - x, y: Coordinates on the specified cubemap face.
    - cubemap_size: Size of each face of the cubemap.

    Returns:
    A tuple (equirect_x, equirect_y) of the equirectangular coordinates.
    """
    

    # Convert cubemap (x, y) to 3D coordinates
    nx = (x / cubemap_size) * 2 - 1
    ny = (y / cubemap_size) * 2 - 1
    if face == 0: # Front
        vector = np.array([nx, -1, ny])
        std = 1.5
    elif face == 1: # Back
        vector = np.array([nx, -1, ny])
        std = 1
    elif face == 2: # Right
        vector = np.array([nx, -1, ny])
        std = 0.5
    elif face == 3: # Left
        vector = np.array([-nx, 1, ny])
        std = 1
    elif face == 4: # Top
        vector = np.array([ny, nx, -1])
        std = 1
    elif face == 5: # Bottom
        vector = np.array([ny, -nx, 1])
        std = 0

    # Normalize vector
    vector = vector / np.linalg.norm(vector)

    # Convert 3D vector to spherical coordinates
    longitude = np.arctan2(vector[1], vector[0])
    latitude = np.arcsin(vector[2])

    # Convert spherical coordinates to equirectangular
    equirect_width = cubemap_size * 4
    equirect_height = cubemap_size * 2
    
    equirect_x = ((longitude + np.pi* std) / (2 * np.pi)+1) * equirect_width % equirect_width
    equirect_y = (latitude + np.pi / 2) / np.pi * equirect_height

    return (equirect_x, equirect_y)

In [29]:
points_sph = []
for idx in range(len(all_points_list)):
    points_tgt = all_points_list[idx]
    for (x_cube, y_cube) in points_tgt:
        #x_sph, y_sph = cube_to_equirectangular(x_cube, y_cube, idx)
        x_sph, y_sph = cubemap_to_equirectangular_coord(idx, x_cube, y_cube)
        x_sph = (int(x_sph + 0.5) + res_std*4) % (res_std*4)
        y_sph = (int(y_sph + 0.5) + res_std*2) % (res_std*2)
        points_sph.append([x_sph, y_sph])
points_sph = np.array(points_sph)

In [30]:
eq_img = cube_to_equirectangular_np(cube_map, res_std*4)
print(eq_img.shape)
if True:
    eq_img = draw_interest_points(eq_img, points_sph)
    #eq_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv.imshow('Random Lines', cv.resize(eq_img, (res_std*4, res_std*2)))
    cv2.imwrite('eq_img.png', eq_img*255)
    cv.waitKey(0)
    cv.destroyAllWindows()

(512, 1024)


In [31]:
points_sph

array([[551, 231],
       [507, 233],
       [515, 214],
       [540, 212],
       [181, 336],
       [150, 316],
       [182, 320],
       [942, 336],
       [938, 323],
       [958, 327],
       [759, 217],
       [727, 228],
       [738, 196],
       [689,  53],
       [505,  79],
       [131,  90],
       [617, 499],
       [234, 467],
       [419, 393]])

In [32]:
#b_img = generate_background(size=(1024, 2048)) /255
##cv.imshow('Random Lines', b_img)
#cv.waitKey(0)
#cv.destroyAllWindows()

In [33]:
#result_image_ndarray = np.where(eq_img*255 == 254, b_img, eq_img)
#result = cv2.resize(result_image_ndarray, (1024, 512))
#cv.imshow('Random Lines', result)
#cv2.imwrite('result.png', result*255)
#cv.waitKey(0)
#cv.destroyAllWindows()